In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap
import seaborn as sns
import string
import pickle # save data frame (results) in a .pkl file
import pandas as pd
from datetime import datetime
import os, sys
import re


sys.path.insert(0, 'C:/Users/User/Code/MMDGraph')
#from importlib import reload
from mmdutils import readfoldertopanda, PlotROCGeneral, plotVaryingBGDEG, findAUC
#reload(readfoldertopanda)
import importlib
importlib.reload(sys.modules['mmdutils'])
from mmdutils import readfoldertopanda, PlotROCGeneral, plotVaryingBGDEG, findAUC

In [22]:
np.random.seed(seed=42)
k = 5
sparsity = .5
# generate the symmetric sparsity mask
mask = np.random.uniform(size = k)
mask = mask * (mask < sparsity)
mask = np.triu(mask)
mask = mask + mask.T + np.identity(k)
mask[mask > 0] = 1

# generate the symmetric precision matrix
theta = np.random.normal(size = (k,k))
theta = np.random.normal(size = (k,k))
theta = np.triu(theta)
theta = theta + theta.T + np.identity(k)

# apply the reqired sparsity
theta = theta * mask

l, _ = np.linalg.eigh(theta)
# force it to be positive definite
theta = theta - (np.min(l)-.1) * np.identity(k)
alpha = 0.0

In [23]:
Gs1 = []
Gs2 = []
error_1 = []
error_2 = []
n = 50

for i in range(50):
    x1 = np.random.multivariate_normal(mean = np.zeros(k), cov = theta, size = 20)
    A1 = np.corrcoef(x1.T)
    if alpha == 0:
        np.fill_diagonal(A1, 0)
    else:
        gl = graphical_lasso(A1, alpha = alpha, max_iter = 1000)
        A1 = gl[0]
        np.fill_diagonal(A1, 0)

    Gs1.append(nx.from_numpy_matrix(A1))
    error_1.append(np.sum(np.logical_xor(np.abs(np.triu(A1,1)) > 0,np.abs(np.triu(theta,1)) > 0)))
    x2 = np.random.multivariate_normal(mean = np.zeros(k), cov = theta, size = 20)
    A2 = np.corrcoef(x2.T)
    if alpha == 0:
        np.fill_diagonal(A2, 0)
    else:
        gl = graphical_lasso(A2, alpha = alpha, max_iter = 1000)
        A2 = gl[0]
        np.fill_diagonal(A2, 0)
    Gs2.append(nx.from_numpy_matrix(A2))
    error_2.append(np.sum(np.logical_xor(np.abs(np.triu(A2,1)) > 0,np.abs(np.triu(theta,1)) > 0)))


In [28]:
x1 = np.random.multivariate_normal(mean = np.zeros(k), cov = theta, size = 20)
A1 = np.corrcoef(x1.T)
if alpha == 0:
    np.fill_diagonal(A1, 0)
else:
    gl = graphical_lasso(theta, alpha = alpha, max_iter = 1000)
    A1 = gl[0]
    np.fill_diagonal(A1, 0)

In [29]:
A1

array([[ 0.        , -0.03047195, -0.05260113,  0.31315765,  0.5948361 ],
       [-0.03047195,  0.        ,  0.09676378,  0.00353337, -0.31479294],
       [-0.05260113,  0.09676378,  0.        , -0.05093755, -0.47208045],
       [ 0.31315765,  0.00353337, -0.05093755,  0.        ,  0.1116744 ],
       [ 0.5948361 , -0.31479294, -0.47208045,  0.1116744 ,  0.        ]])

In [27]:
A1

array([[ 0.        ,  0.24125776, -0.01447986,  0.04410383,  0.21007179],
       [ 0.24125776,  0.        , -0.18524236,  0.47480949,  0.32925484],
       [-0.01447986, -0.18524236,  0.        ,  0.44316404, -0.81285048],
       [ 0.04410383,  0.47480949,  0.44316404,  0.        , -0.18926471],
       [ 0.21007179,  0.32925484, -0.81285048, -0.18926471,  0.        ]])

In [30]:
theta

array([[ 3.09415253,  0.        ,  0.        ,  0.        ,  0.38934891],
       [ 0.        ,  5.60164783,  0.        ,  0.        ,  0.21863832],
       [ 0.        ,  0.        ,  0.25107488,  0.        , -0.53814166],
       [ 0.        ,  0.        ,  0.        ,  3.68652107,  0.58212279],
       [ 0.38934891,  0.21863832, -0.53814166,  0.58212279,  2.17070852]])

In [14]:
A1

array([[ 0.        ,  0.02057183, -0.10274722, -0.03133482,  0.32833986,
        -0.00847185, -0.21464823,  0.09137159, -0.01238261,  0.04469932,
        -0.26454124],
       [ 0.02057183,  0.        ,  0.11706564,  0.1534338 , -0.29505998,
         0.25023394,  0.46669416, -0.08162329,  0.07115306, -0.05571581,
        -0.14649726],
       [-0.10274722,  0.11706564,  0.        ,  0.07545947, -0.31040235,
         0.11138671,  0.37122399, -0.22186608,  0.21436664, -0.04152643,
        -0.29093666],
       [-0.03133482,  0.1534338 ,  0.07545947,  0.        , -0.32148477,
        -0.22005376,  0.4067953 , -0.03441569,  0.04311767,  0.11917227,
        -0.35269385],
       [ 0.32833986, -0.29505998, -0.31040235, -0.32148477,  0.        ,
         0.11847436, -0.13018082,  0.13955166, -0.04669534,  0.08680246,
        -0.00123082],
       [-0.00847185,  0.25023394,  0.11138671, -0.22005376,  0.11847436,
         0.        , -0.00742658, -0.08556496, -0.03353983, -0.12888968,
        -0.251

In [5]:
data = pd.read_pickle('../data/GLasso/alpha_0.0.pkl')

In [6]:
data

,p_val,sample,mean_error_1,mean_error_2,alpha
0,1.0000,0,30.0,30.0,0.0
1,0.9999,1,30.0,30.0,0.0
2,1.0000,2,30.0,30.0,0.0
3,0.9999,3,30.0,30.0,0.0
4,1.0000,4,30.0,30.0,0.0
...,...,...,...,...,...
245,1.0000,245,30.0,30.0,0.0
246,1.0000,246,30.0,30.0,0.0
247,1.0000,247,30.0,30.0,0.0
248,1.0000,248,30.0,30.0,0.0
